In [1]:
import sys 
sys.path.append("/Users/chilpert/Work/pyproteinsExt/src")
sys.path.append("/Users/chilpert/Work/pyproteins/src")

In [2]:
import pickle, time
import time

def save(data, tag=None):
    saveDir="/Volumes/arwen/mobi/group/NOX_CH/pickle_saved"
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fTag = "NOX_annotation_" + tag + "_" if tag else "NOX_annotation_"
    fSerialDump = fTag + timestr + ".pickle"
    with open(saveDir + '/' + fSerialDump, 'wb') as f:
        pickle.dump(data, f)
    print('data structure saved to', saveDir + '/' + fSerialDump)

def load(fileName):
    saveDir="/Volumes/arwen/mobi/group/NOX_CH/pickle_saved"
    d = pickle.load( open(saveDir + "/" + fileName, "rb" ) )
    print("restore a annotated container of ", len(d), "elements")
    return d

#### Load data

In [3]:
data=load("NOX_annotation_noEukaryota_noObsolete_20190516-172955.pickle")

restore a annotated container of  163 elements


### Get proteins sequences

In [109]:
def load_sequences(fasta_file):
    dic_seq={}
    f=open(fasta_file)
    for l in f : 
        if l.startswith(">"):
            ref=l.split(" ")[0].lstrip(">")
            dic_seq[ref]=''
        else: 
            dic_seq[ref]+=l.rstrip()
    return dic_seq        

In [116]:
dic_seq=load_sequences("/Volumes/arwen/mobi/group/NOX_CH/seedSet/NOX_noEukaryota.mfasta")

### Get profiles sequences

In [4]:
dic_profile={'PF08022_full':{},'PF01794_full':{},'PF08030_full':{}}
for p in data :
    for profile in data[p]['hmmr']: 
        if len(data[p]['hmmr'][profile][0].data)!=1: 
            print(p, "has more than one hit for", profile, "handle it !!!")
            break 
        hit=data[p]['hmmr'][profile][0].data[0]
        seq=hit.aliStringLetters
        seq=seq.replace("-","")
        seq=''.join(x.upper() for x in seq)
        start=int(hit.aliFrom)
        end=int(hit.aliTo)
        dic_profile[profile][p]={'seq':seq,'start':start,'end':end}

In [10]:
def write_fasta_profile_seq(dic_profile,output_dir):
    for profile in dic_profile: 
        fasta=open(output_dir+"/"+profile+"_prokaryotes_seq.fasta","w")
        for p in dic_profile[profile]:
            header=">"+p+" "+profile+"\n"
            seq=dic_profile[profile][p]['seq']+"\n"
            fasta.write(header+seq)
        fasta.close()    

In [12]:
write_fasta_profile_seq(dic_profile,"/Volumes/arwen/mobi/group/NOX_CH/AffineProfile")

```console 
for p in PF01794_full PF08022_full PF08030_full; 
do clustalw2 -INFILE=$p\_prokaryotes_seq.fasta -OUTFILE=$p\_multiple_alignment.clustal;
hmmbuild -n $p\_prokaryotes $p\_prokaryotes.hmm $p\_multiple_alignment.clustal ;
done
```